### Train a machine learning model 

#### Step 1: Setup 

In [1]:
#%pip install pandas numpy -qU

In [2]:
#%pip install scikit-learn -qU

In [3]:
# %pip install shap -qU

In [4]:
 #%pip install xgboost -qU

In [5]:
# %pip install  matplotlib -qU

In [6]:
 #%pip install numpy -qU

In [7]:
# %pip install catboost -qU

In [8]:
# %pip install lightgbm -qU

In [9]:
# %pip install seaborn -qU

In [10]:
# %pip install optuna -qU

In [11]:
# !nvidia-smi

In [12]:
import pandas as pd
import os
import sklearn
import shap
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_validate,RandomizedSearchCV, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from trainMLmodel import surfaceDesignTopAndBottom,selectDescriptors,combineRankAndDescriptorsClassyfication,explainer, combineRankAndDescriptorsClassificationbotisFLAT, combineRankAndDescriptorsClassyfication_Danilingus
import numpy as np

In [13]:
cd

/bme001.mnt/home/20212358


#### Step 2: load the data file

In [14]:
# Define the location of the csv files imageOutliersRemoved and Ranking
#pathToImageObject=os.getcwd()+"/DataAnalysis/"
pathToImageObject="/bme001.mnt/home/20212358"
locationOfImageObject1="cleaned_rawImagedataFI_final.csv"
locationOfImageObject2="DataAnalysis_withJan/Screen_Mean_Count_CellBody_new.csv"
# Define the location of the annotation files
pathToTopoUnitImages="/FeatureImages/FeatureImages/"
allTopoDescriptors=pd.read_csv("allTopoDescriptors.csv")
# check if file exists
if os.path.isfile(locationOfImageObject1):
    imagedataOutliersRemoved=pd.read_csv(locationOfImageObject1,low_memory=False)
    if os.path.isfile(locationOfImageObject2):
        imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)
    else:
        print("File not found, check if you finished ranking the features")
else:
    print('File not find, check if you removed the outliers')


In [15]:
#imagedataOutliersRemoved["Mean_Nephrin_Granularity_8_CorrCellBody"]

In [16]:
#replacing ranking by previous module with ranking sorted from lowest hit to the highest
#locationOfImageObject2=pathToImageObject+"/DataAnalysis/Screen_Mean_Mean_Nephrin_AreaShape_MajorAxisLength.csv"
# locationOfImageObject2=pathToImageObject+"/DataAnalysis/PostBlindTestHits/Screen_Mean_Mean_Nephrin_Granularity_8_CorrCellBody.csv"
# imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)

In [17]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Count_CellBody_new  \
0            710                                0.25   
1           1877                                0.25   
2           1448                                0.25   
3           1076                                0.25   
4           1054                                0.25   
...          ...                                 ...   
1829        1288                                0.25   
1830        1742                                0.25   
1831         836                                0.25   
1832        1292                                0.25   
1833        1625                                0.25   

      Screen_KSpvalue_Count_CellBody_new  Screen_SNR_Count_CellBody_new  \
0                               0.474026                       3.799227   
1                               0.930736                       5.293229   
2                               0.962704                       3.913095   
3                               0.962704                       3.102444   
4                               0.660140                       2.805779   
...                                  ...                            ...   
1829                            0.637363                       2.343262   
1830                            0.660140                       2.257996   
1831                            0.735664                       2.389780   
1832                            0.316239                       2.175057   
1833                            1.000000                       2.795554   

      Screen_Mean_Count_CellBody_new  Screen_SD_Count_CellBody_new  \
0                          47.833333                     12.590280   
1                          47.583333                      8.989472   
2                          46.785714                     11.956193   
3                          46.428571                     14.965161   
4                          46.125000                     16.439282   
...                              ...                           ...   
1829                       24.571429                     10.485993   
1830                       24.466667                     10.835568   
1831                       24.000000                     10.042766   
1832                       23.384615                     10.751267   
1833                       19.833333                      7.094599   

      Screen_Median_Count_CellBody_new    Rank  
0                                 49.5     1.0  
1                                 47.5     2.0  
2                                 49.5     3.0  
3                                 53.5     4.0  
4                                 53.0     5.0  
...                                ...     ...  
1829                              23.0  1830.0  
1830                              24.0  1831.0  
1831                              21.0  1832.0  
1832                              22.0  1833.0  
1833                              18.5  1834.0  

[1834 rows x 8 columns]

In [18]:
# locationOfImageObject3=pathToImageObject+"/DataAnalysis/Screen_Mean_Count_CellBody_new_Ranking.csv"
# imageDataRankedCount=pd.read_csv(locationOfImageObject3,low_memory=False)
# imageDataRankedCount

#### Step 3: Select Features for the machine learning.

##### Step 3.1 Vizualize the topographies design from top and bottom of the rank.

First, select the feature of interest

In [19]:
featureOfInterest='Screen_Mean_Count_FilteredNuclei_new'

Second, select the number of top and bottom ranked feautures you want to select

In [20]:
numberOfRankedFeaturesToVisualize=100

In [21]:
#surfaceDesignTopAndBottom(imageDataRankedSurfaces,pathToTopoUnitImages,featureOfInterest,number=numberOfRankedFeaturesToVisualize)
imagedataOutliersRemoved

AreaOccupied_AreaOccupied_CellBody_new  \
0                                    195441.0   
1                                    190993.0   
2                                    157466.0   
3                                    128276.0   
4                                    159374.0   
...                                       ...   
31915                                146459.0   
31916                                113561.0   
31917                                136466.0   
31918                                108352.0   
31919                                132796.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_new  \
0                                           41149.0   
1                                           34146.0   
2                                           21593.0   
3                                           30686.0   
4                                           36365.0   
...                                             ...   
31915                                       28944.0   
31916                                       21965.0   
31917                                       22229.0   
31918                                       18080.0   
31919                                       35925.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_target  \
0                                              38157.0   
1                                              33484.0   
2                                              19752.0   
3                                              28584.0   
4                                              35031.0   
...                                                ...   
31915                                          24587.0   
31916                                          19472.0   
31917                                          18386.0   
31918                                          12223.0   
31919                                          27669.0   

       AreaOccupied_AreaOccupied_MaskedNuclei  \
0                                    147590.0   
1                                    178264.0   
2                                    112684.0   
3                                    115514.0   
4                                    147877.0   
...                                       ...   
31915                                128723.0   
31916                                 96719.0   
31917                                109961.0   
31918                                 73657.0   
31919                                106458.0   

       AreaOccupied_AreaOccupied_Nephrin  AreaOccupied_Perimeter_CellBody_new  \
0                               166158.0                              17129.0   
1                               210194.0                              18661.0   
2                               133496.0                              13267.0   
3                               138323.0                              11218.0   
4                               200466.0                              16275.0   
...                                  ...                                  ...   
31915                           165853.0                              13291.0   
31916                           153661.0                              10378.0   
31917                           143216.0                              12602.0   
31918                           146310.0                              10716.0   
31919                           133671.0                              10817.0   

       AreaOccupied_Perimeter_FilteredNuclei_new  \
0                                         6242.0   
1                                         5837.0   
2                                         3797.0   
3                                         4632.0   
4                                         5380.0   
...                                          ...   
31915                                     4030.0   
31916                                     3448.0   
31917                         

### Step 4 Load and vizualize descriptors data

##### Step 4.1 select descriptors based on a keyword 

In [22]:
selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
             # keywordsToInclude=["Kamiel"],
                 # keywordsToExclude=["_Index","Height","Width", "Center","Texture","ImageNumber","Zernike","sd","Count","Number","var","AngleBetweenNeighbors","mad","Neighbors"])
#selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
              #  keywordsToInclude=["Pattern","Kamiel"],
                 #keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike"])
   keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike","Texture"])

In [23]:
pd.options.display.max_columns = 30
selectedDescriptors

FeatureIdx  Pattern_AreaOccupied_AreaOccupied_Pattern  \
0              1                                       5215   
1              2                                       2604   
2              3                                      20366   
3              4                                       4864   
4              5                                      17246   
...          ...                                        ...   
2171        2172                                       3821   
2172        2173                                      16244   
2173        2174                                       1855   
2174        2175                                       4794   
2175        2176                                      23548   

      Pattern_AreaOccupied_Perimeter_Pattern  \
0                                        330   
1                                        422   
2                                        910   
3                                        439   
4                                       1252   
...                                      ...   
2171                                     436   
2172                                    1497   
2173                                     306   
2174                                     492   
2175                                    1632   

      Pattern_AreaOccupied_TotalArea_Pattern  Pattern_Count_Pattern  \
0                                      40000                      1   
1                                      10000                      1   
2                                      40000                      1   
3                                      10000                      1   
4                                      40000                      1   
...                                      ...                    ...   
2171                                   10000                      1   
2172                                   78400                      2   
2173                                   10000                      1   
2174                                   10000                      2   
2175                                   78400                      1   

      Pattern_AreaShape_Area_sd  Pattern_AreaShape_Compactness_sd  \
0                         0.000                          0.000000   
1                         0.000                          0.000000   
2                         0.000                          0.000000   
3                         0.000                          0.000000   
4                         0.000                          0.000000   
...                         ...                               ...   
2171                      0.000                          0.000000   
2172                   3183.395                          0.654076   
2173                      0.000                          0.000000   
2174                   1681.500                          0.306309   
2175                      0.000                          0.000000   

      Pattern_AreaShape_Eccentricity_sd  Pattern_AreaShape_Extent_sd  \
0                              0.000000                     0.000000   
1                              0.000000                     0.000000   
2                              0.000000                     0.000000   
3                              0.000000                     0.000000   
4                              0.000000                     0.000000   
...                                 ...                          ...   
2171                           0.000000                     0.000000   
2172                           0.052437                     0.022692   
2173                           0.000000                     0.000000   
2174                           0.553171                     0.158168   
2175                           0.000000                     0.000000   

      Pattern_AreaShape_FormFactor_sd  Pattern_AreaShape_MajorAxisLength_sd  \
0                            0.000000         

In [24]:
Screen_Mean_Mean_Nephrin_Granularity_1 = pd.read_csv("Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody.csv")
Screen_Mean_Mean_Nephrin_Granularity_1

FeatureIdx  Screen_ADpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0           1658                                           0.250000         
1            189                                           0.083959         
2           1270                                           0.095665         
3            122                                           0.250000         
4           1353                                           0.250000         
...          ...                                                ...         
2086        1836                                           0.163023         
2087        1860                                           0.250000         
2088        1394                                           0.250000         
2089         757                                           0.069970         
2090        1292                                           0.250000         

      Screen_KSpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              0.730111         
1                                              0.126748         
2                                              0.055944         
3                                              0.647552         
4                                              0.700123         
...                                                 ...         
2086                                           0.078568         
2087                                           0.860428         
2088                                           0.730111         
2089                                           0.078568         
2090                                           0.893706         

      Screen_SNR_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              5.508653    
1                                              5.717872    
2                                              4.253026    
3                                              5.190216    
4                                              4.866214    
...                                                 ...    
2086                                           5.657096    
2087                                           5.332652    
2088                                           5.167565    
2089                                           6.501527    
2090                                           3.432248    

      Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                             46.026999     
1                                             45.273450     
2                                             45.259933     
3                                             45.068389     
4                                             44.444154     
...                                                 ...     
2086                                          30.269535     
2087                                          29.868709     
2088                                          29.775861     
2089                                          29.053096     
2090                                          28.751350     

      Screen_SD_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              8.355400   
1                                              7.917885   
2                                             10.641819   
3                                              8.683335   
4                                              9.133211   
...                                                 ...   
2086                                           5.350720   
2087                                           5.601099   
2088                                           5.762068   
2089                                           4.468657   
2090                                           8.376829   

      Screen_Median_Mean_Nephrin_Granularity_1_CorrCellBody    Rank  
0                                             45.283829         1.0

### Step 5: Train Machine Learning models

##### Step 5.1 : train the model based on the ranked surfaces and the descriptors selected in step 4.1

In [25]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Count_CellBody_new  \
0            710                                0.25   
1           1877                                0.25   
2           1448                                0.25   
3           1076                                0.25   
4           1054                                0.25   
...          ...                                 ...   
1829        1288                                0.25   
1830        1742                                0.25   
1831         836                                0.25   
1832        1292                                0.25   
1833        1625                                0.25   

      Screen_KSpvalue_Count_CellBody_new  Screen_SNR_Count_CellBody_new  \
0                               0.474026                       3.799227   
1                               0.930736                       5.293229   
2                               0.962704                       3.913095   
3                               0.962704                       3.102444   
4                               0.660140                       2.805779   
...                                  ...                            ...   
1829                            0.637363                       2.343262   
1830                            0.660140                       2.257996   
1831                            0.735664                       2.389780   
1832                            0.316239                       2.175057   
1833                            1.000000                       2.795554   

      Screen_Mean_Count_CellBody_new  Screen_SD_Count_CellBody_new  \
0                          47.833333                     12.590280   
1                          47.583333                      8.989472   
2                          46.785714                     11.956193   
3                          46.428571                     14.965161   
4                          46.125000                     16.439282   
...                              ...                           ...   
1829                       24.571429                     10.485993   
1830                       24.466667                     10.835568   
1831                       24.000000                     10.042766   
1832                       23.384615                     10.751267   
1833                       19.833333                      7.094599   

      Screen_Median_Count_CellBody_new    Rank  
0                                 49.5     1.0  
1                                 47.5     2.0  
2                                 49.5     3.0  
3                                 53.5     4.0  
4                                 53.0     5.0  
...                                ...     ...  
1829                              23.0  1830.0  
1830                              24.0  1831.0  
1831                              21.0  1832.0  
1832                              22.0  1833.0  
1833                              18.5  1834.0  

[1834 rows x 8 columns]

In [26]:
imageDataRankedSurfaces.shape

(1834, 8)

In [27]:
# #pd.merge(left=imageDataRankedSurfaces, right=result_df,
#                       how='left', left_on="FeatureIdx", 
#                       right_on="FeatureIdx").Rank_x

In [28]:
X,Y=combineRankAndDescriptorsClassyfication(SortedRank=imageDataRankedSurfaces,Descriptors=selectedDescriptors,numberPerClass=90)

X.shape

(180, 383)

In [29]:
Y=Y.astype(bool)

In [30]:
def reduce_memory_usage(df):
    import gc
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()
                
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)

                elif str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)
                    
                if(df[col].nunique()>100):
                    df[col] = df[col].astype('category')
                else:
                    if((col_type != 'datetime64[ns]')):
                        df[col] = df[col].astype(str)
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage became: ",mem_usg," MB")
    gc.collect()
    return df

X=reduce_memory_usage(X)

Memory usage of dataframe is 0.53 MB
Memory usage became:  0.13556289672851562  MB


##### Step 5.2: split X and Y in training and test data set

In [31]:
a=0
b=0
for i in Y.values:
    if (i==False):
        b+=1
    else:
        a+=1
a,b

(90, 90)

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y )
# X_train,X_val,y_train,y_val=train_test_split(X_train,y_train)

In [33]:
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.2,stratify=y_test )


##### Step 5.3: select a classification model

In [34]:
import lightgbm as lgb
import joblib
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score


def objective(trial,X_train =X_train ,y_train = y_train, X_test = X_test,y_test=y_test, big_df = X, big_y=Y ):

    param = {
    'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart','rf']), #, 'rf' 
    'num_leaves': trial.suggest_int('num_leaves', 2, 2500),
    'max_depth':trial.suggest_int('max_depth', -10, 900),
    'objective':trial.suggest_categorical('objective', ['binary']),
    'learning_rate': trial.suggest_float('learning_rate', 1e-12, 0.99, log=False),
    'n_estimators': trial.suggest_int('n_estimators', 100, 24_100),
    #'subsample_for_bin': trial.suggest_int('subsample_for_bin', 50_000, 450_000, 50_000),
    'random_state': trial.suggest_categorical('random_state', [42]),
    'data_sample_strategy':trial.suggest_categorical('data_sample_strategy', ['bagging','goss']),
    # 'tree_learner':trial.suggest_categorical('tree_learner', ['serial', 'feature', 'data', 'voting']),
    'deterministic':trial.suggest_categorical('deterministic', [True]),
    'force_col_wise':trial.suggest_categorical('force_col_wise', [True,False]),
    # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 2_230),
    
    'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf',  0, 100),
# #     'bagging_fraction': trial.suggest_float('bagging_fraction',  1e-12, 1),
    'feature_fraction': trial.suggest_float('feature_fraction', 1e-12, 0.9, log=False),
    'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 1e-12, 0.9, log=False),
    'extra_trees':trial.suggest_categorical('extra_trees', [True,False]),
    'lambda_l1': trial.suggest_float('lambda_l1',  0, 100),
    'lambda_l2': trial.suggest_float('lambda_l2',  0, 100),
    'linear_lambda': trial.suggest_float('linear_lambda',  0, 100),
    'drop_rate': trial.suggest_float('drop_rate',  0, 1),
    'max_drop': trial.suggest_int('max_drop', 0, 2000),
    'skip_drop': trial.suggest_float('skip_drop',  0, 1),
    'uniform_drop':trial.suggest_categorical('uniform_drop', [True,False]),
    'top_rate': trial.suggest_float('top_rate',  0, 0.5),
    'other_rate': trial.suggest_float('other_rate',  0, 0.5),
    'min_data_per_group': trial.suggest_int('min_data_per_group', 1, 2000),
    'max_cat_threshold': trial.suggest_int('max_cat_threshold', 1, 2000),
    'cat_l2': trial.suggest_float('cat_l2',  0, 100),
    'cat_smooth': trial.suggest_float('cat_smooth',  0, 100),
    'top_k': trial.suggest_int('top_k', 1, 2000),
    'cegb_tradeoff': trial.suggest_float('cegb_tradeoff',  0, 100),
    # 'cegb_penalty_split': trial.suggest_float('cegb_penalty_split',  0, 100),
    # 'path_smooth': trial.suggest_float('path_smooth',  0, 100),
    # 'num_grad_quant_bins': trial.suggest_int('num_grad_quant_bins', 1, 2000),
#     'is_unbalance':trial.suggest_categorical('uniform_drop', [True,False]),
    'scale_pos_weight': trial.suggest_float('scale_pos_weight',  1e-12, 100),
    'sigmoid': trial.suggest_float('sigmoid',  1e-12, 100),
    # 'boost_from_average':trial.suggest_categorical('boost_from_average', [True,False]),
    'max_bin':trial.suggest_int('max_bin', 50, 1_000),
        
    'n_jobs':trial.suggest_categorical('n_jobs', [-1]), 
    'verbose': trial.suggest_categorical('verbose', [-1]), 
    }


    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])


    y_pred_test = model.predict(X_test)
    # print(y_pred_test)
    

    #accuracy_test = balanced_accuracy_score(y_pred_test, y_test)

    #accuracy_test = (sklearn.model_selection.cross_val_score (model, big_df, big_y, cv=5, scoring='balanced_accuracy', n_jobs=1)).mean()
    # try:
    #     accuracy_test = roc_auc_score(y_pred_test, y_test)
    # except Exception:
    #     accuracy_test=0

    try:
        predicted_train = model.predict(X_train)
        probabilities_train = model.predict_proba(X_train)
        predicted_test = model.predict(X_test)
        probabilities_test = model.predict_proba(X_test)
            
        train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
        test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
    
        accuracy_train = metrics.auc(train_fpr, train_tpr)
        accuracy_test = metrics.auc(test_fpr, test_tpr)

        bal_accuracy_train = balanced_accuracy_score(predicted_train, y_train)
        bal_accuracy_test = balanced_accuracy_score(predicted_test, y_test) 
    except Exception:
        accuracy_test=0
        accuracy_train = 0
        bal_accuracy_test = 0
        bal_accuracy_train=0


    joblib.dump(study, 'another_lgbm_with_val.pkl')

    if (int(trial.number)%50==0):
        print("BACKUP GOES BRRRRR")
        joblib.dump(study, 'another_lgbm_with_val_BACKUP.pkl')


    return accuracy_test, accuracy_train,bal_accuracy_train,bal_accuracy_test

In [35]:
# https://vaex.io/docs/tutorial_ml.html#CatBoost-example

In [36]:
def logging_callback(study, frozen_trial):
    # Retrieve previous best values for both objectives
    previous_best_value_0 = study.user_attrs.get("previous_best_value_0", None)
    previous_best_value_1 = study.user_attrs.get("previous_best_value_1", None)
    previous_best_value_2 = study.user_attrs.get("previous_best_value_2", None)
    previous_best_value_3 = study.user_attrs.get("previous_best_value_3", None)

    
    # Get the current trial's values
    current_values = frozen_trial.values

    if current_values is None:
        return  # Skip if the trial doesn't have valid results

    # current_value_0, current_value_1 = current_values
    current_value_0, current_value_1,current_value_2,current_value_3 = current_values


    # Check if both objectives improved
    if (
        (previous_best_value_0 is None or current_value_0 >= previous_best_value_0) and
        (previous_best_value_1 is None or current_value_1 >= previous_best_value_1) and
        (previous_best_value_2 is None or current_value_2 >= previous_best_value_2) and
        (previous_best_value_3 is None or current_value_3 >= previous_best_value_3) 
    ):
        # Update the best values in the study's user attributes
        study.set_user_attr("previous_best_value_0", current_value_0)
        study.set_user_attr("previous_best_value_1", current_value_1)
        study.set_user_attr("previous_best_value_2", current_value_2)
        study.set_user_attr("previous_best_value_3", current_value_3)

        # Log the improved metrics
        print(40*"*",)
        print("\n\n\n")
        print(
            f"\nNew best trial with both objectives improved!\n"
            f"Trial {frozen_trial.number}:\n"
            f"  Objective 0: {current_value_0}\n"
            f"  Objective 1: {current_value_1}\n"
            f"  Objective 2: {current_value_2}\n"
            f"  Objective 3: {current_value_3}\n"
            f"  Params: {frozen_trial.params}\n"
        )
        print("\n\n\n")

        print(40*"*","\n")


In [37]:
#!rm another_lgbm_with_val.pkl

In [38]:
!free -th

              total        used        free      shared  buff/cache   available
Mem:          251Gi        16Gi       156Gi       2.7Gi        78Gi       230Gi
Swap:         3.7Gi        67Mi       3.7Gi
Total:        255Gi        16Gi       160Gi


In [39]:
#plotting SHAP

import shap
import matplotlib.pyplot as plt
import joblib
import lightgbm as lgb

# Load the Optuna study
study = joblib.load('another_lgbm_with_val.pkl')  # Ensure the correct path

# Retrieve the best trial from the Pareto front
best_trial = study.best_trials[0]  # Select the first Pareto-optimal trial
best_params = best_trial.params

print("Best Parameters:", best_params)

# Load the dataset (ensure these variables are defined in your notebook)
X_train = X_train  # Replace with actual training data variable
y_train = y_train  # Replace with actual training labels variable
X_test = X_test    # Replace with actual testing data variable

# Train the LightGBM model with the best parameters
model = lgb.LGBMClassifier(**best_params)  # Use LGBMRegressor if regression
model.fit(X_train, y_train)

# Use TreeExplainer for LightGBM models
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_test)

# Plot and save the SHAP summary plot
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig("shap_summary_plot.png", bbox_inches='tight')
print("SHAP summary plot saved as 'shap_summary_plot.png'")


Best Parameters: {'boosting_type': 'gbdt', 'num_leaves': 2344, 'max_depth': 143, 'objective': 'binary', 'learning_rate': 0.160839184175993, 'n_estimators': 19096, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 24.510336946654864, 'feature_fraction': 0.800147235827867, 'feature_fraction_bynode': 0.13100195956003666, 'extra_trees': True, 'lambda_l1': 81.11613026633972, 'lambda_l2': 63.050402187691866, 'linear_lambda': 91.92050676719862, 'drop_rate': 0.5015053380297326, 'max_drop': 661, 'skip_drop': 0.7721270831877045, 'uniform_drop': False, 'top_rate': 0.11268141057481107, 'other_rate': 0.27760236901042024, 'min_data_per_group': 896, 'max_cat_threshold': 1258, 'cat_l2': 92.67257503587795, 'cat_smooth': 85.79697472853832, 'top_k': 615, 'cegb_tradeoff': 55.539820505123096, 'scale_pos_weight': 1.60063435484666, 'sigmoid': 71.71476934052164, 'max_bin': 113, 'n_jobs': -1, 'verbose': -1}


/home/20212358/.local/lib/python3.10/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


SHAP summary plot saved as 'shap_summary_plot.png'


In [40]:
import joblib

# Load Optuna study
study = joblib.load('another_lgbm_with_val.pkl')

# Get the best trial parameters
best_trial = study.best_trials[0]
best_params = best_trial.params

# Train the model with the best parameters
from lightgbm import LGBMClassifier
bestClassyficationModel = LGBMClassifier(**best_params)
bestClassyficationModel.fit(X_train, y_train)


from lightgbm import plot_importance
import matplotlib.pyplot as plt

# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 10))
plot_importance(bestClassyficationModel, max_num_features=15, ax=ax, importance_type='split')
plt.title("Top 15 Features by Importance")

plt.show()
plt.savefig("Top 15 Features by Importance", bbox_inches='tight')

In [39]:
%%time 


#optuna.logging.set_verbosity(optuna.logging.WARNING)

import optuna, gc
gc.collect()

if os.path.isfile('another_lgbm_with_val.pkl'):
    print("Found file !")
    study = joblib.load('another_lgbm_with_val.pkl')
else:
    print("CREATING NEW FILE...")
    sampler = optuna.samplers.TPESampler(multivariate=True, warn_independent_sampling=False)
    study = optuna.create_study(directions=['maximize', 'maximize','maximize','maximize'], load_if_exists=True,sampler=sampler,pruner=optuna.pruners.MedianPruner(n_warmup_steps=30,n_startup_trials=20))
study.optimize(objective, timeout=1_000_000, gc_after_trial=True,
              show_progress_bar=True, n_jobs=1, callbacks=[logging_callback])
print("Finished working")
joblib.dump(study, 'another_lgbm_with_val.pkl')

Found file !


   0%|          | 00:00/277:46:40

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-01 12:54:07,555] Trial 977 finished with values: [0.5357142857142857, 0.4442515432098766, 0.5851063829787233, 0.5] and parameters: {'boosting_type': 'dart', 'num_leaves': 1478, 'max_depth': 545, 'objective': 'binary', 'learning_rate': 0.2599104916381412, 'n_estimators': 12106, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 61.6580269272404, 'feature_fraction': 0.7544763338793374, 'feature_fraction_bynode': 0.7333475816912371, 'extra_trees': False, 'lambda_l1': 38.3277818510238, 'lambda_l2': 16.5860440165025, 'linear_lambda': 45.01296728052699, 'drop_rate': 0.3622799290928092, 'max_drop': 1971, 'skip_drop': 0.4441057165467875, 'uniform_drop': True, 'top_rate': 0.1632073029071851, 'other_rate': 0.2128028856713458, 'min_data_per_group': 141, 'max_cat_threshold': 302, 'cat_l2': 44.18196863807294, 'cat_smooth': 4.8621236181394565, 'top_k': 1349, 'cegb_tradeoff': 3.805905199098269, 'scale_pos_weight': 

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


...

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-01 13:03:32,902] Trial 1077 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'rf', 'num_leaves': 155, 'max_depth': 104, 'objective': 'binary', 'learning_rate': 0.8651018871890418, 'n_estimators': 1401, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 37.49350165334014, 'feature_fraction': 0.7297976902082923, 'feature_fraction_bynode': 0.7464989175110002, 'extra_trees': False, 'lambda_l1': 92.0384965157897, 'lambda_l2': 51.34996616623456, 'linear_lambda': 38.95611604954751, 'drop_rate': 0.8660540891622265, 'max_drop': 1783, 'skip_drop': 0.2338235045690421, 'uniform_drop': False, 'top_rate': 0.19361083398359658, 'other_rate': 0.15422749090696392, 'min_data_per_group': 1757, 'max_cat_threshold': 352, 'cat_l2': 14.78451692675359, 'cat_smooth': 0.4664075338628815, 'top_k': 229, 'cegb_tradeoff': 5.456830635002762, 'scale_pos_weight': 60.476921716869896, 'sigmoid': 33.909991

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-01 13:03:45,960] Trial 1079 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'dart', 'num_leaves': 806, 'max_depth': 170, 'objective': 'binary', 'learning_rate': 0.13528065778414522, 'n_estimators': 9044, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 76.67230835654985, 'feature_fraction': 0.6129029671818613, 'feature_fraction_bynode': 0.774339038308447, 'extra_trees': False, 'lambda_l1': 97.25415401688358, 'lambda_l2': 85.82772594855368, 'linear_lambda': 13.072305834281629, 'drop_rate': 0.3952057870644018, 'max_drop': 1606, 'skip_drop': 0.2150333577538226, 'uniform_drop': True, 'top_rate': 0.07687768989827448, 'other_rate': 0.2811440828531165, 'min_data_per_group': 1170, 'max_cat_threshold': 1334, 'cat_l2': 46.86139590009176, 'cat_smooth': 40.436315542696505, 'top_k': 355, 'cegb_tradeoff': 35.96418884998986, 'scale_pos_weight': 66.9850001182493, 'sigmoid': 60.692877

KeyboardInterrupt: 

In [ ]:
print(1/0)
#CHECK TRIAL 1580, 1709, 1845, 4669, 9338

#after second december 2024
#check 643 (0.17,0.94);  765 (0.16, 1.0);  801 (0.154, 1.0);

In [40]:
study = joblib.load('another_lgbm_with_val.pkl')



In [41]:
# trial_number = 4669
# specific_trial = next((t for t in study.trials if t.number == trial_number), None)

# if specific_trial is not None:
#     print(f"Trial #{specific_trial.number}")
#     print("Parameters:", specific_trial.params)
#     print("Value:", specific_trial.values)
# else:
#     print(f"Trial #{trial_number} not found.")

In [42]:
print('Number of finished trials:', len(study.trials))
try:
    print('Best trial:', study.best_trial.params)
except Exception:
    pass

Number of finished trials: 1081


In [43]:
try:
    Best_trial = study.best_trial.params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)
except Exception:
    Best_trial = study.best_trials[0].params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)

{'boosting_type': 'gbdt', 'num_leaves': 1843, 'max_depth': 687, 'objective': 'binary', 'learning_rate': 0.3305901690206237, 'n_estimators': 11663, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 12.137168778393807, 'feature_fraction': 0.7112870517845802, 'feature_fraction_bynode': 0.827482987601983, 'extra_trees': False, 'lambda_l1': 68.41797089293752, 'lambda_l2': 9.736889640105513, 'linear_lambda': 69.34109901532379, 'drop_rate': 0.19794073585196287, 'max_drop': 1643, 'skip_drop': 0.2390831989124672, 'uniform_drop': True, 'top_rate': 0.16597924554861446, 'other_rate': 0.39700507357525927, 'min_data_per_group': 610, 'max_cat_threshold': 484, 'cat_l2': 19.121696772353104, 'cat_smooth': 16.888611775175782, 'top_k': 1858, 'cegb_tradeoff': 56.86450195415614, 'scale_pos_weight': 11.189144848892113, 'sigmoid': 41.61194896938733, 'max_bin': 778, 'n_jobs': -1, 'verbose': -1}


In [44]:
for i in range(len(study.best_trials)):
    print(study.best_trials[i].values)

[0.6683673469387754, 1.0, 1.0, 0.7512820512820513]
[0.7244897959183674, 1.0, 1.0, 0.71875]
[0.4489795918367347, 0.9529320987654322, 0.7903225806451613, 0.7692307692307692]
[0.75, 0.9143518518518519, 0.8111111111111111, 0.7857142857142857]
[0.75, 0.9211033950617284, 0.806033589052457, 0.7857142857142857]
[0.7448979591836735, 0.9174382716049383, 0.8130434782608695, 0.7512820512820513]
[0.7346938775510203, 0.9288194444444445, 0.8279705369257608, 0.7857142857142857]
[0.8010204081632653, 0.9027777777777778, 0.8031275060144346, 0.6555555555555556]
[0.7397959183673468, 0.9305555555555555, 0.8010033444816054, 0.7857142857142857]
[0.7755102040816326, 0.9074074074074074, 0.8142857142857143, 0.6555555555555556]
[0.7602040816326532, 0.9120370370370371, 0.7653562653562653, 0.5933333333333333]
[0.7448979591836735, 0.923804012345679, 0.8010033444816054, 0.7857142857142857]
[0.75, 0.9220679012345679, 0.8010033444816054, 0.7857142857142857]
[0.7397959183673469, 0.9263117283950617, 0.8111111111111111, 0

In [61]:
try:
    print(study.best_trial.value)
except Exception:
    print(study.best_trials[2].values)

[0.4489795918367347, 0.9529320987654322, 0.7903225806451613, 0.7692307692307692]


###### study.best_trials[1].params

In [81]:
%%capture

"""
try:
    best_model = lgb.LGBMClassifier(**study.best_params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])
except Exception:
    print("Using alternative")
    best_model = lgb.LGBMClassifier(**study.best_trials[0].params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])

"""

best_model = lgb.LGBMClassifier(**study.best_trials[3].params)
#best_model = lgb.LGBMClassifier(**specific_trial.params)
best_model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])

In [82]:
# best_model = cb.CatBoostClassifier(**study.best_params,
#                                 ).fit(X, Y, verbose=False, early_stopping_rounds=500)

In [83]:
best_model.predict(X_test)

array([False,  True, False,  True,  True,  True,  True,  True, False,
        True, False, False,  True,  True, False, False, False, False,
       False, False,  True,  True, False,  True, False,  True,  True,
       False])

In [84]:
predicted_train = best_model.predict(X_train)
probabilities_train = best_model.predict_proba(X_train)
predicted_test = best_model.predict(X_test)
probabilities_test = best_model.predict_proba(X_test)

#FOR VALIDATION
predicted_val = best_model.predict(X_val)
probabilities_val = best_model.predict_proba(X_val)

In [85]:
y_test.nunique()

2

In [86]:
from sklearn.metrics import balanced_accuracy_score

display(balanced_accuracy_score(y_test, predicted_test))
display(balanced_accuracy_score(y_train, predicted_train))

0.7857142857142857

0.7916666666666666

In [87]:
# FOR VALIDATION

display(balanced_accuracy_score(y_val, predicted_val))

0.75

In [88]:
# calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
#obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
print(train_roc_auc, test_roc_auc)
#make the plot
fig, ax = plt.subplots(figsize=[5,5])
plt.title('Receiver Operating Characteristic')
plt.plot(train_fpr, train_tpr, 'b', label = 'Train AUC = %0.2f' % train_roc_auc)
plt.plot(test_fpr, test_tpr, 'g', label = 'Test AUC = %0.2f' % test_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('another_lgbm_with_val.png')
plt.show()


0.9143518518518519 0.75


In [133]:
#-------------------

In [134]:
#together with Koen
bestClassyficationModel=RandomForestClassifier(**bestParameters)
bestClassyficationModel.fit(X_train, y_train)

NameError: name 'RandomForestClassifier' is not defined

In [135]:
predicted_train = bestClassyficationModel.predict(X_train)
probabilities_train = bestClassyficationModel.predict_proba(X_train)
predicted_test = bestClassyficationModel.predict(X_test)
probabilities_test = bestClassyficationModel.predict_proba(X_test)

NameError: name 'bestClassyficationModel' is not defined

Step 5.7 :Print out accuracy metrics

In [ ]:
print ('\nTrain Accuracy:', metrics.accuracy_score(y_train, predicted_train))
print ('Test Accuracy:', metrics.accuracy_score(y_test, predicted_test))


Step 5.8: plot the AUC curve for training and test data

In [ ]:
# calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
#obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
#make the plot
fig, ax = plt.subplots(figsize=[5,5])
plt.title('Receiver Operating Characteristic')
plt.plot(train_fpr, train_tpr, 'b', label = 'Train AUC = %0.2f' % train_roc_auc)
plt.plot(test_fpr, test_tpr, 'g', label = 'Test AUC = %0.2f' % test_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Step 5.9: Plot 10 most important features for the model

In [ ]:
plot_importance(bestClassyficationModel,max_num_features=20,xlabel='Features weight');    
plt.show()

### Step 6: Use SHAP module to explain predicted model

##### Step 6.1: Show which features contribute to prediction of particular surface

In [ ]:
explainerDefinition=explainer(dataForModel=X,
          model=bestClassyficationModel,
          hit=1)

##### Step 6.2: Show SHAP values for every important feature and every predicted surface Plot explanation https://shap.readthedocs.io/en/latest/

In [ ]:
shap_values = explainerDefinition.shap_values(X)
shap.summary_plot(shap_values, X)

##### Step 6.3: Draw a scatter plot with features of interest

Set the feature of interest for the x- and y-axis

In [ ]:
xFeature="Kamiel_FCP"
yFeature='InsCirclRadius_median'

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])

colors = {0:'orange', 1:'blue'}

binned=X
binned["Class"]=Y

grouped = binned.groupby("Class")

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', 
               x=xFeature, 
               y=yFeature, 
               label=key, color=colors[key])

plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# 1. Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 2. Scale Features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# 3. Hyperparameter Search Space Refinement
searchParameters = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.0001, 0.001, 0.01],
    'n_estimators': [100, 250, 500],
    'reg_alpha': [0.01, 0.1, 1],
    'reg_lambda': [0.01, 0.1, 1]
}

# 4. Updated Fitting Parameters
fittingParameters = {
    "early_stopping_rounds": 50,
    "eval_metric": "auc",
    "eval_set": [(X_test_scaled, y_test)]  # Using validation set for early stopping
}

# 5. Classification Model
classificationModel = XGBClassifier(objective='binary:logistic', random_state=1)

# 6. Randomized Search with Cross-Validation
random_search = RandomizedSearchCV(
    estimator=classificationModel,
    param_distributions=searchParameters,
    n_iter=50,  # Reduce iterations for quicker testing; increase later
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=10,
    random_state=1
)

# 7. Fit the Model
searchModel = random_search.fit(X_resampled_scaled, y_resampled, **fittingParameters)

# 8. Best Parameters
bestParameters = searchModel.best_estimator_.get_params()
print("Best Parameters:", bestParameters)

# 9. Evaluate Model on Test Data
y_pred = searchModel.predict(X_test_scaled)
test_auc = roc_auc_score(y_test, y_pred)
train_auc = roc_auc_score(y_resampled, searchModel.predict(X_resampled_scaled))

print(f"Train AUC: {train_auc}")
print(f"Test AUC: {test_auc}")
